#### Install the useful packages.

In [1]:
myPackages <- c("easypackages")
#the argument `lib.loc = "~/rlib"` is used because the packages were installed in this folder in my case
lapply(myPackages, library, lib.loc = "~/rlib", character.only = TRUE) 
libraries("dplyr", "tidyr")

[[1]]
[1] "easypackages" "stats"        "graphics"     "grDevices"    "utils"       
[6] "datasets"     "methods"      "base"

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr

All packages loaded successfully



#### Import the gallup data that contain every country.

In [2]:
gallupRaw <- rio::import("~/gallup data/The_Gallup_042722.dat")

Ambiguous file format ('.dat'), but attempting 'data.table::fread("~/gallup data/The_Gallup_042722.dat")'



We need country name, country code, weight, survey date, survey year, ls, hope, PA, NA, gender, age, employment, marriage, region

Filter the data and save the it for future use.

In [ ]:
gallupExtracted <- gallupRaw %>% select(COUNTRYNEW, COUNTRY_ISO3, WGT, 
                                        FIELD_DATE, WP4.INTERVIEWDATE = WP4, YEAR_WAVE, 
                                        YEAR_CALENDAR, ls = WP16, hope = WP18, 
                                        smile = WP63, enjoy = WP67, worry = WP69, 
                                        sad = WP70, stress = WP71, anger = WP74,
                                        WP1219.MALE = WP1219, WP1220.AGE = WP1220, EMP_2010,
                                        WP1223.MARITAL = WP1223, starts_with("REGION_"))
write.csv(gallupExtracted, "gallupExtracted.csv", row.names = FALSE)

In [27]:
gallupExtracted <- rio::import("gallupExtracted.csv")
colnames(gallupExtracted)
head(gallupExtracted)
dim(gallupExtracted)

[1] "COUNTRYNEW"        "COUNTRY_ISO3"      "WGT"              
  [4] "FIELD_DATE"        "WP4.INTERVIEWDATE" "YEAR_WAVE"        
  [7] "YEAR_CALENDAR"     "ls"                "hope"             
 [10] "smile"             "enjoy"             "worry"            
 [13] "sad"               "stress"            "anger"            
 [16] "WP1219.MALE"       "WP1220.AGE"        "EMP_2010"         
 [19] "WP1223.MARITAL"    "REGION_AFG"        "REGION_AGO"       
 [22] "REGION_ALB"        "REGION_ARE"        "REGION_ARG"       
 [25] "REGION_ARM"        "REGION_AUS"        "REGION_AUT"       
 [28] "REGION_AZE"        "REGION_BDI"        "REGION_BEL"       
 [31] "REGION_BEN"        "REGION_BFA"        "REGION_BGD"       
 [34] "REGION_BGR"        "REGION_BHR"        "REGION_BIH"       
 [37] "REGION_BLR"        "REGION_BLZ"        "REGION_BOL"       
 [40] "REGION_BRA"        "REGION_BTN"        "REGION_BWA"       
 [43] "REGION_CAM"        "REGION_CAN"        "REGION_CAR"       
 [46] "REGION_CHE"        "REGION_CHL"        "REGION_CHN"       
 [49] "REGION_CIV"        "REGION_COG"        "REGION_COL"       
 [52] "REGION_COM"        "REGION_CRI"        "REGION_CYP"       
 [55] "REGION_CZE"        "REGION_DEU"        "REGION_DJI"       
 [58] "REGION_DOM"        "REGION_DRC"        "REGION_DZA"       
 [61] "REGION_ECU"        "REGION_EGY"        "REGION_ESP"       
 [64] "REGION_EST"        "REGION_ETH"        "REGION_FIN"       
 [67] "REGION_FRA"        "REGION_GAB"        "REGION_GBR"       
 [70] "REGION_GEO"        "REGION_GHA"        "REGION_GMB"       
 [73] "REGION_GRC"        "REGION_GTM"        "REGION_GUI"       
 [76] "REGION_HKG"        "REGION_HND"        "REGION_HRV"       
 [79] "REGION_HTI"        "REGION_HUN"        "REGION_ICL"       
 [82] "REGION_IDN"        "REGION_IND"        "REGION_IRL"       
 [85] "REGION_IRN"        "REGION_IRQ"        "REGION_ISR"       
 [88] "REGION_ITA"        "REGION_JMC"        "REGION_JOR"       
 [91] "REGION_JPN"        "REGION_KAZ"        "REGION_KEN"       
 [94] "REGION_KGZ"        "REGION_KHM"        "REGION_KOR"       
 [97] "REGION_KOS"        "REGION_KWT"        "REGION_LAO"       
[100] "REGION_LBN"        "REGION_LBR"        "REGION_LKA"       
[103] "REGION_LSO"        "REGION_LTU"        "REGION_LUX"       
[106] "REGION_LVA"        "REGION_LYB"        "REGION_MAR"       
[109] "REGION_MDA"        "REGION_MDG"        "REGION_MEX"       
[112] "REGION_MKD"        "REGION_MLI"        "REGION_MLT"       
[115] "REGION_MLW"        "REGION_MMR"        "REGION_MNE"       
[118] "REGION_MNG"        "REGION_MOZ"        "REGION_MRT"       
[121] "REGION_MUS"        "REGION_MYS"        "REGION_NAM"       
[124] "REGION_NCY"        "REGION_NGA"        "REGION_NIC"       
[127] "REGION_NKR"        "REGION_NLD"        "REGION_NOR"       
[130] "REGION_NPL"        "REGION_NZL"        "REGION_OMN"       
[133] "REGION_PAK"        "REGION_PAN"        "REGION_PER"       
[136] "REGION_PHL"        "REGION_POL"        "REGION_PRI"       
[139] "REGION_PRT"        "REGION_PRY"        "REGION_PSE"       
[142] "REGION_QAT"        "REGION_ROU"        "REGION_RUS"       
[145] "REGION_RWA"        "REGION_SAU"        "REGION_SDN"       
[148] "REGION_SEN"        "REGION_SLE"        "REGION_SLV"       
[151] "REGION_SOL"        "REGION_SOM"        "REGION_SRB"       
[154] "REGION_SRI"        "REGION_SSD"        "REGION_SVK"       
[157] "REGION_SVN"        "REGION_SWE"        "REGION_SWZ"       
[160] "REGION_SYR"        "REGION_TCD"        "REGION_THA"       
[163] "REGION_TJK"        "REGION_TKM"        "REGION_TOG"       
[166] "REGION_TTO"        "REGION_TUN"        "REGION_TUR"       
[169] "REGION_TWN"        "REGION_TZA"        "REGION_UKR"       
[172] "REGION_URY"        "REGION_USA"        "REGION_UZB"       
[175] "REGION_VEN"        "REGION_VNM"        "REGION_YEM"       
[178] "REGION_ZAF"        "REGION_ZWE"

,COUNTRYNEW,COUNTRY_ISO3,WGT,FIELD_DATE,WP4.INTERVIEWDATE,YEAR_WAVE,YEAR_CALENDAR,ls,hope,smile,⋯,REGION_TZA,REGION_UKR,REGION_URY,REGION_USA,REGION_UZB,REGION_VEN,REGION_VNM,REGION_YEM,REGION_ZAF,REGION_ZWE
,<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,United States,USA,0.6689163,7/1/2006,,2006,2006,6,8,1,⋯,NA,NA,NA,23,NA,NA,NA,NA,NA,NA
2,United States,USA,0.1966555,7/1/2006,,2006,2006,8,98,1,⋯,NA,NA,NA,55,NA,NA,NA,NA,NA,NA
3,United States,USA,1.3032635,7/1/2006,,2006,2006,8,8,1,⋯,NA,NA,NA,48,NA,NA,NA,NA,NA,NA
4,United States,USA,0.6359364,7/1/2006,,2006,2006,8,8,1,⋯,NA,NA,NA,12,NA,NA,NA,NA,NA,NA
5,United States,USA,0.7358092,7/1/2006,,2006,2006,10,10,1,⋯,NA,NA,NA,47,NA,NA,NA,NA,NA,NA
6,United States,USA,1.7369582,7/1/2006,,2006,2006,5,10,1,⋯,NA,NA,NA,18,NA,NA,NA,NA,NA,NA


[1] 2450403     179

Create a function for changing the "DK" / "Refused" response to NA.

In [5]:
na_codes <- function(x, ...) {
    for(y in list(...))
        x[x == y] <- NA
    x
}

Change the "DK" / "Refused" response to NA

Change WP1219.MALE from [1 Male, 2 Female] to [1 Yes, 0 No]

Change the responses to SWB from [1 Yes, 2 No] to [1 Yes, 0 No] for getting the means for individual positive affect and negative affect

Collapse REGION columns into one column and add country code as a prefix.

In [28]:
gallupAssas <- gallupExtracted

gallupAssas$WP1220.AGE <-  na_codes(gallupAssas$WP1220.AGE, 100)
gallupAssas$WP1223.MARITAL <-  na_codes(gallupAssas$WP1223.MARITAL, 6,7)

#WP1219.MALE
gallupAssas$WP1219.MALE <- 2 - gallupAssas$WP1219.MALE

#convert empty WP4.INTERVIEWDATE to NA
gallupAssas <- gallupAssas %>% mutate(WP4.INTERVIEWDATE = na_if(WP4.INTERVIEWDATE, "")) 
#life satisfication
gallupAssas$ls <- na_codes(gallupAssas$ls, 98, 99)
#hope
gallupAssas$hope <- na_codes(gallupAssas$hope, 98, 99)
#pa and #na
gallupAssas$smile <- 2 - na_codes(gallupAssas$smile, 3, 4)
gallupAssas$enjoy <- 2 - na_codes(gallupAssas$enjoy, 3, 4)
gallupAssas$worry <- 2 - na_codes(gallupAssas$worry, 3, 4)
gallupAssas$sad <- 2 - na_codes(gallupAssas$sad, 3, 4)
gallupAssas$stress <- 2 - na_codes(gallupAssas$stress, 3, 4)
gallupAssas$anger <- 2 - na_codes(gallupAssas$anger, 3, 4)
gallupAssas <- gallupAssas %>% 
 rowwise() %>% 
 mutate(pa = mean(cbind(enjoy, smile), na.rm = TRUE),
        na = mean(cbind(worry, sad, stress, anger), na.rm = TRUE)) %>% ungroup
head(gallupAssas)

COUNTRYNEW,COUNTRY_ISO3,WGT,FIELD_DATE,WP4.INTERVIEWDATE,YEAR_WAVE,YEAR_CALENDAR,ls,hope,smile,⋯,REGION_URY,REGION_USA,REGION_UZB,REGION_VEN,REGION_VNM,REGION_YEM,REGION_ZAF,REGION_ZWE,pa,na
<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
United States,USA,0.6689163,7/1/2006,NA,2006,2006,6,8,1,⋯,NA,23,NA,NA,NA,NA,NA,NA,1,0.0
United States,USA,0.1966555,7/1/2006,NA,2006,2006,8,NA,1,⋯,NA,55,NA,NA,NA,NA,NA,NA,1,0.0
United States,USA,1.3032635,7/1/2006,NA,2006,2006,8,8,1,⋯,NA,48,NA,NA,NA,NA,NA,NA,1,0.5
United States,USA,0.6359364,7/1/2006,NA,2006,2006,8,8,1,⋯,NA,12,NA,NA,NA,NA,NA,NA,1,0.0
United States,USA,0.7358092,7/1/2006,NA,2006,2006,10,10,1,⋯,NA,47,NA,NA,NA,NA,NA,NA,1,0.0
United States,USA,1.7369582,7/1/2006,NA,2006,2006,5,10,1,⋯,NA,18,NA,NA,NA,NA,NA,NA,1,1.0


In [61]:
gallupAssas2 <- gallupAssas
regionColnames <- colnames(gallupAssas2 %>% select(starts_with("REGION_")))
gallupAssas2$REGION_ALL <- coalesce(!!! gallupAssas2 %>% select(starts_with("REGION_")))
gallupAssas2$REGION_ALL <- paste0(gallupAssas2$COUNTRY_ISO3, "_", gallupAssas2$REGION_ALL)
gallupAssas2 <- gallupAssas2 %>% select(-all_of(regionColnames))
head(gallupAssas2)

COUNTRYNEW,COUNTRY_ISO3,WGT,FIELD_DATE,WP4.INTERVIEWDATE,YEAR_WAVE,YEAR_CALENDAR,ls,hope,smile,⋯,sad,stress,anger,WP1219.MALE,WP1220.AGE,EMP_2010,WP1223.MARITAL,pa,na,REGION_ALL
<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>
United States,USA,0.6689163,7/1/2006,NA,2006,2006,6,8,1,⋯,0,0,0,0,47,NA,2,1,0.0,USA_23
United States,USA,0.1966555,7/1/2006,NA,2006,2006,8,NA,1,⋯,0,0,0,0,67,NA,5,1,0.0,USA_55
United States,USA,1.3032635,7/1/2006,NA,2006,2006,8,8,1,⋯,0,1,0,1,62,NA,2,1,0.5,USA_48
United States,USA,0.6359364,7/1/2006,NA,2006,2006,8,8,1,⋯,0,0,0,1,82,NA,5,1,0.0,USA_12
United States,USA,0.7358092,7/1/2006,NA,2006,2006,10,10,1,⋯,0,0,0,0,62,NA,2,1,0.0,USA_47
United States,USA,1.7369582,7/1/2006,NA,2006,2006,5,10,1,⋯,1,1,1,0,32,NA,2,1,1.0,USA_18


In [ ]:
write.csv(gallupAssas2, "gallupAssas.csv", row.names = FALSE)

In [62]:
colnames(gallupAssas2)
dim(gallupAssas2)

[1] "COUNTRYNEW"        "COUNTRY_ISO3"      "WGT"              
 [4] "FIELD_DATE"        "WP4.INTERVIEWDATE" "YEAR_WAVE"        
 [7] "YEAR_CALENDAR"     "ls"                "hope"             
[10] "smile"             "enjoy"             "worry"            
[13] "sad"               "stress"            "anger"            
[16] "WP1219.MALE"       "WP1220.AGE"        "EMP_2010"         
[19] "WP1223.MARITAL"    "pa"                "na"               
[22] "REGION_ALL"

[1] 2450403      22